In [1]:
from typing import List, Mapping
from itertools import zip_longest, chain
from glob import iglob
from pickle import load
from functools import partial, reduce

from scipy.ndimage.filters import gaussian_filter
import pandas as pd
import numpy as np
from numpy.random import normal, choice, rand
from numpy.polynomial.legendre import Legendre
import matplotlib.pyplot as plt
from h5py import File

from vmitools import (
    abel_inverse, finite_legendre_transform_in_theta,
    interp, tohist, transpose_linearly, transpose_to_drdomega, transpose_to_drdth,
    mrot, mhorshear, msqueeze,
)

In [14]:
runs = [333, 334, 335, 342]
filenames = sorted(chain.from_iterable(iglob(f"/data/*/Run_{r:03d}/work/reduced.pickle") for r in runs if not r in {}))
print(f"Total {len(filenames)} files:")
for fn in filenames:
    print(f"    {fn}")

Total 4 files:
    /data/Step601N2/Run_333/work/reduced.pickle
    /data/Step601N2/Run_334/work/reduced.pickle
    /data/Step601N2/Run_335/work/reduced.pickle
    /data/Step601N2/Run_342/work/reduced.pickle


In [15]:
def read_file(filename):
    with open(filename, 'br') as f:
        return load(f)


sumup = partial(reduce, partial(pd.DataFrame.add, fill_value=0))
summed = sumup(read_file(fn) for fn in filenames)
summed

sum   count
is_bg delay atmax                                                           
False -7.0  510    [[110.0, 102.0, 109.0, 106.0, 112.0, 106.0, 11...     1.0
            550    [[105.0, 105.0, 98.0, 107.0, 107.0, 101.0, 102...     1.0
            555    [[520.0, 510.0, 512.0, 543.0, 524.0, 522.0, 50...     5.0
            560    [[18270.0, 18160.0, 18245.0, 18355.0, 18502.0,...   174.0
            565    [[159215.0, 158557.0, 158313.0, 160230.0, 1608...  1511.0
            570    [[383009.0, 379958.0, 380284.0, 384934.0, 3866...  3627.0
            575    [[371546.0, 368095.0, 369144.0, 373593.0, 3749...  3527.0
            580    [[146016.0, 144591.0, 145204.0, 146530.0, 1473...  1396.0
            585    [[17862.0, 17651.0, 17770.0, 17960.0, 17951.0,...   172.0
            590    [[3982.0, 3846.0, 3949.0, 4000.0, 3994.0, 3961...    38.0
            595    [[2200.0, 2193.0, 2185.0, 2208.0, 2222.0, 2186...    21.0
            600    [[2949.0, 2912.0, 2919.0, 2949.0, 2941.0, 2902...    28.0
      -6.9  560    [[742.0, 753.0, 725.0, 738.0, 752.0, 737.0, 74...     7.0
            565    [[15028.0, 15084.0, 14906.0, 15189.0, 15250.0,...   143.0
            570    [[107301.0, 106441.0, 106577.0, 108352.0, 1086...  1017.0
            575    [[215478.0, 213523.0, 214012.0, 216865.0, 2176...  2046.0
            580    [[143149.0, 141741.0, 142575.0, 143777.0, 1444...  1368.0
            585    [[23063.0, 22976.0, 22999.0, 23302.0, 23452.0,...   222.0
            590    [[3916.0, 3938.0, 3916.0, 3966.0, 3987.0, 3933...    38.0
            595    [[102.0, 103.0, 104.0, 103.0, 108.0, 102.0, 97...     1.0
      -6.8  555    [[108.0, 102.0, 108.0, 106.0, 105.0, 104.0, 11...     1.0
            560    [[1368.0, 1364.0, 1348.0, 1374.0, 1372.0, 1347...    13.0
            565    [[855.0, 847.0, 839.0, 836.0, 851.0, 837.0, 82...     8.0
            570    [[4161.0, 4108.0, 4127.0, 4206.0, 4168.0, 4106...    39.0
            575    [[27320.0, 27170.0, 27050.0, 27439.0, 27520.0,...   258.0
            580    [[50818.0, 50086.0, 50356.0, 50847.0, 51212.0,...   484.0
            585    [[18346.0, 18196.0, 18281.0, 18396.0, 18466.0,...   176.0
            590    [[2125.0, 2039.0, 2077.0, 2092.0, 2101.0, 2069...    20.0
      -6.7  560    [[202.0, 205.0, 197.0, 210.0, 214.0, 207.0, 20...     2.0
            565    [[1471.0, 1472.0, 1451.0, 1496.0, 1493.0, 1451...    14.0
...                                                              ...     ...
True  -6.3  565    [[212.0, 212.0, 202.0, 212.0, 208.0, 204.0, 19...     2.0
            570    [[844.0, 803.0, 829.0, 856.0, 851.0, 836.0, 82...     8.0
            575    [[6968.0, 6869.0, 6875.0, 6960.0, 7022.0, 6936...    67.0
            580    [[28431.0, 28011.0, 28090.0, 28532.0, 28548.0,...   274.0
            585    [[9334.0, 9210.0, 9209.0, 9340.0, 9416.0, 9263...    90.0
            590    [[1856.0, 1826.0, 1857.0, 1865.0, 1873.0, 1835...    18.0
      -6.2  560    [[741.0, 709.0, 727.0, 739.0, 749.0, 729.0, 70...     7.0
            565    [[310.0, 316.0, 317.0, 315.0, 315.0, 309.0, 32...     3.0
            570    [[1142.0, 1161.0, 1136.0, 1148.0, 1166.0, 1143...    11.0
            575    [[8243.0, 8091.0, 8184.0, 8255.0, 8297.0, 8170...    79.0
            580    [[26270.0, 25786.0, 26191.0, 26309.0, 26430.0,...   253.0
            585    [[10182.0, 10006.0, 10096.0, 10168.0, 10212.0,...    98.0
            590    [[2380.0, 2417.0, 2382.0, 2399.0, 2384.0, 2380...    23.0
      -6.1  545    [[98.0, 105.0, 103.0, 103.0, 106.0, 102.0, 100...     1.0
            555    [[105.0, 105.0, 105.0, 103.0, 110.0, 106.0, 10...     1.0
            560    [[1546.0, 1559.0, 1553.0, 1565.0, 1557.0, 1521...    15.0
            565    [[1043.0, 1040.0, 1033.0, 1051.0, 1037.0, 1020...    10.0
            570    [[732.0, 701.0, 733.0, 734.0, 737.0, 733.0, 73...     7.0
            575    [[6694.0, 6648.0, 6585.0, 6695.0, 6748.0, 6640...    64.0
            580    [[28329.0, 27897.0, 28175.0, 28477.0, 2847

In [ ]:
def good_img(img):
    return not np.isnan(img).any()


n = summed['count'].loc[False] + summed['count'].loc[True]
avg = summed['sum'] / summed['count']
diff = (avg.loc[False] - avg.loc[True])
where = (50 < n) & (diff.apply(good_img))
df = pd.DataFrame({
    'img': diff[where].apply(partial(np.einsum, 'ij->ji')),
    'n': n[where],
})

plt.figure(figsize=(8, 16))
for i, ((dt, atmax), img, n) in enumerate(df[['img', 'n']].itertuples()):
    plt.subplot(10, 3, i+1)
    plt.title(f"{dt:.3f} ps\npeak at {atmax} px\n{n} counts")
    plt.pcolormesh(img.T, cmap='terrain')
    plt.axis("equal")
    plt.xticks([], [])
    plt.yticks([], [])
    plt.clim(0, None)
plt.tight_layout()
plt.show()

ValueError: num must be 1 <= num <= 30, not 31

In [ ]:
zedges = np.arange(-400, 400 + 1)
redges, thedges = np.linspace(0, 400, 400 + 1), np.linspace(-np.pi, np.pi, 360 + 1)
r = (redges[1:] + redges[:-1]) / 2
th = (thedges[1:] + thedges[:-1]) / 2


def invert_img(d):
    img = d['img']
    img = gaussian_filter(img, 2)
    xn, yn = img.shape
    xedges = np.arange(xn + 1)
    yedges = np.arange(yn + 1)
    f = interp(img, xedges, yedges)
    g = transpose_linearly(f, np.eye(2), x0=np.array([437, 458]))
    transformed = tohist(g, zedges, zedges)

    dz = zedges[1:] - zedges[:-1]
    inverted = abel_inverse(transformed, zedges) * dz[None, :]  # shape: (r, z)
    sliced = interp(inverted, zedges, zedges)  # (rho, z) -> intensity
    hist_indrdomega = tohist(transpose_to_drdomega(sliced), redges, thedges)  # (r, th) -> intensity
    hist_indrdth = tohist(transpose_to_drdth(sliced), redges, thedges)  # (r, th) -> intensity
    return pd.Series({
        'inverted': inverted,
        'hist_indrdth': hist_indrdth,
        'hist_indrdomega': hist_indrdomega,
    })


df = df.merge(df.apply(invert_img, axis=1), left_index=True, right_index=True)


plt.figure(figsize=(8, 16))
for i, ((dt, atmax), img, n) in enumerate(df[['inverted', 'n']].itertuples()):
    plt.subplot(6, 3, i+1)
    plt.title(f"{dt:.3f} ps, peak at {atmax} px")
    plt.pcolormesh(img.T, cmap='terrain')
    plt.axis("equal")
    plt.xticks([], [])
    plt.yticks([], [])
    plt.clim(0, None)
plt.tight_layout()
plt.show()

In [ ]:
roi = [
    [0, 100],
    [130, 190],
    [230, 265],
    [275, 300],
    [305, 325],
    [330, 350],
]


def f(hist):
    return hist.sum(1)


def intergrateit(hist, fr, to, x=None, **kwargs):
    if x is None:
        x = np.arange(len(hist))
    where = (fr < x) & (x < to)
    return hist[where].sum(**kwargs)


def project_to_pn(hist, n):
    _, coeff, _ = finite_legendre_transform_in_theta(
        hist, thedges, n + 1,
    )  # Shapes of returns: (n, r) (n, r) (n, th)
    return coeff[n]/coeff[0]


for i, (fr, to) in enumerate(roi):
    df[f'summed{i}_rdist'] = df['hist_indrdth'].apply(intergrateit, fr=fr, to=to, x=r)
    df[f'summed{i}_pad'] = df['hist_indrdomega'].apply(intergrateit, fr=fr, to=to, x=r, axis=0)
    df[f'summed{i}_beta2'] = df[f'summed{i}_pad'].apply(project_to_pn, n=2)
    df[f'summed{i}_beta4'] = df[f'summed{i}_pad'].apply(project_to_pn, n=4)


plt.figure(figsize=(12, 6))
plt.subplot(121)
for (dt, atmax), dist in df['hist_indrdth'].apply(f).items():
    plt.plot(r, dist, label=f"delay={dt:.3f},atmax={atmax}")
plt.minorticks_on()
plt.grid(True, which='both')

for fr, to in roi:
    plt.axvspan(fr, to, alpha=0.2)

plt.subplot(122)
for (dt, atmax), dist in df['hist_indrdth'].apply(f).items():
    plt.plot(r**2, dist/2/r, label=f"delay={dt:.3f}, atmax={atmax}")
plt.minorticks_on()
plt.grid(True, which='both')

for fr, to in roi:
    plt.axvspan(fr**2, to**2, alpha=0.2)

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 16))
for i, ((dt, atmax), pad) in enumerate(df['summed2_pad'].items()):
    plt.subplot(6, 3, i+1)
    plt.title(f"{dt:.3f} ps\npeak at {atmax} px")
    plt.plot(th, pad)
    plt.xticks([], [])
    plt.yticks([0], [0])
    plt.ylim(0, None)
plt.tight_layout()
plt.show()

In [ ]:
at = df.index[0][0]


def norm(arr):
    return arr/arr.sum()


plt.figure()
plt.xlabel("Pixel at maximum")
plt.ylabel(r"$\beta_2$")
plt.plot(df.loc[at]['summed0_beta2'], label='A0')
plt.plot(df.loc[at]['summed1_beta2'], label='X4')
plt.plot(df.loc[at]['summed2_beta2'], label='X2')
plt.plot(df.loc[at]['summed3_beta2'], label='X1')
plt.plot(df.loc[at]['summed4_beta2'], label='X0')
plt.plot(df.loc[at]['summed4_beta2'], label='Unkown')
# plt.ylim(0, None)
plt.grid(True)
plt.figlegend()
plt.tight_layout()
plt.show()

In [ ]:
at = df.index[0][0]


def norm(arr):
    return arr/arr.sum()


plt.figure()
plt.xlabel("Pixel at maximum")
plt.ylabel(r"$\beta_4$")
plt.plot(df.loc[at]['summed0_beta4'], label='A0')
plt.plot(df.loc[at]['summed1_beta4'], label='X4')
plt.plot(df.loc[at]['summed2_beta4'], label='X2')
plt.plot(df.loc[at]['summed3_beta4'], label='X1')
plt.plot(df.loc[at]['summed4_beta4'], label='X0')
plt.plot(df.loc[at]['summed5_beta4'], label='Unkown')
# plt.ylim(0, None)
plt.grid(True)
plt.figlegend()
plt.tight_layout()
plt.show()

In [ ]:
at = df.index[0][0]


def norm(arr):
    return arr/arr.sum()


plt.figure()
plt.subplot(121)
plt.xlabel("Pixel at maximum")
plt.ylabel("Photoelectron yield")
plt.plot(df.loc[at]['summed0_rdist'], label='A0')
plt.plot(df.loc[at]['summed1_rdist'], label='X4')
plt.plot(df.loc[at]['summed2_rdist'], label='X2')
plt.plot(df.loc[at]['summed3_rdist'], label='X1')
plt.plot(df.loc[at]['summed4_rdist'], label='X0')
plt.plot(df.loc[at]['summed5_rdist'], label='Unkown')
plt.ylim(0, None)
plt.grid(True)
plt.figlegend()

plt.subplot(122)
plt.xlabel("Pixel at maximum")
plt.ylabel("Normalized photoelectron yield")
plt.plot(norm(df.loc[at]['summed0_rdist']), label='A0')
plt.plot(norm(df.loc[at]['summed1_rdist']), label='X4')
plt.plot(norm(df.loc[at]['summed2_rdist']), label='X2')
plt.plot(norm(df.loc[at]['summed3_rdist']), label='X1')
plt.plot(norm(df.loc[at]['summed4_rdist']), label='X0')
plt.plot(norm(df.loc[at]['summed5_rdist']), label='Unkown')
plt.ylim(0, None)
plt.grid(True)
# plt.figlegend()
plt.tight_layout()
plt.show()